In [1]:
from TutorML.nn import BernouliRBM

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import sklearn.neural_network as nn
from scipy.ndimage import convolve
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# This function is from sklearn user guide: 
# Restricted Boltzmann Machine features for digit classification
def nudge_dataset(X, Y):
    """
    This produces a dataset 5 times bigger than the original one,
    by moving the 8x8 images in X around by 1px to left, right, down, up
    """
    direction_vectors = [
        [[0, 1, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [1, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 1],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 1, 0]]]

    shift = lambda x, w: convolve(x.reshape((8, 8)), mode='constant',
                                  weights=w).ravel()
    X = np.concatenate([X] +
                       [np.apply_along_axis(shift, 1, X, vector)
                        for vector in direction_vectors])
    Y = np.concatenate([Y for _ in range(5)], axis=0)
    return X, Y

# Load Data
digits = datasets.load_digits()
X = np.asarray(digits.data, 'float32')
X, Y = nudge_dataset(X, digits.target)
X = (X - np.min(X, 0)) / (np.max(X, 0) + 0.0001)  # 0-1 scaling

X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    test_size=0.2,
                                                    random_state=0)

### Baseline

In [3]:
lr = LogisticRegression()
lr.fit(X_train,Y_train)
pred = lr.predict(X_test)
print(np.mean(pred==Y_test))

0.7673900946021146


### My RBM

In [8]:
rbm = BernouliRBM(n_visibles=64, n_hiddens=100)
rbm.fit(X_train,batch_size=5,n_epochs=20,learning_rate=0.5)
H_train = rbm.transform(X_train)
H_test = rbm.transform(X_test)

In [9]:
lr = LogisticRegression()
lr.fit(H_train,Y_train)
pred = lr.predict(H_test)
print(np.mean(pred==Y_test))

0.8887033945464663


### sklearn RBM

In [10]:
rbm = nn.BernoulliRBM(n_components=100,learning_rate=0.1)
rbm.fit(X_train)
H_train = rbm.transform(X_train)
H_test = rbm.transform(X_test)

In [11]:
lr = LogisticRegression()
lr.fit(H_train,Y_train)
pred = lr.predict(H_test)
print(np.mean(pred==Y_test))

0.8525319977740679
